# PLS Regression

In [6]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_theme(
    context="paper", 
    style="whitegrid", 
    font_scale=1.2,
    rc={'figure.figsize': (10, 10), 'figure.dpi': 300}
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Get Data

In [7]:
from common.data import get_data

X, Y, demographics, population = get_data()
ages = demographics['Age']

print(f'X: {X.shape} | Y: {len(Y.keys())} | Age: {ages.shape} | Population: {population}')

X: (390, 34716) | Y: 16 | Age: (390,) | Population: adhd


In [8]:
from scipy import stats
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RepeatedKFold, cross_validate, permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

## Run for one target, one age bin

In [28]:
selected_target = "WISC_PSI"
y = Y[selected_target]

print(f'y: {y.shape}')

y: (390,)


In [42]:
from common.binning import bin_by_age

bins = bin_by_age(X, y, ages, y, True)
bin_1, bin_2, bin_3 = bins[0], bins[1], bins[2]
print(f'Bin 1: {bin_1[0].shape} | Bin 2: {bin_2[0].shape} | Bin 3: {bin_3[0].shape}')

Bin 0 Range: 6.22 -> 8.80
Bin 1 Range: 9.34 -> 11.97
Bin 2 Range: 12.50 -> 15.87
---
Bin 0 Range: 80.00 -> 144.00
Bin 1 Range: 86.00 -> 123.00
Bin 2 Range: 66.00 -> 126.00
---
Bin 1: (19, 34716) | Bin 2: (16, 34716) | Bin 3: (16, 34716)


### Set up the model pipeline, metrics, and cross-validation approach

In [14]:
def regression_scorer(reg, X, y):
    y_pred = reg.predict(X)[:, 0]
    return stats.pearsonr(y, y_pred)[0]

estimators = [StandardScaler(), PLSRegression(n_components=4)]
pipe = make_pipeline(*estimators)

scoring = ['train_score', 'test_score']
rkf = RepeatedKFold(n_splits=10, n_repeats=10)
n_perm = 3000

### Run cross-validation

In [29]:
%%time
# X_cv = bin_3[0]
# y_cv = bin_3[1]
X_cv = X
y_cv = y
age_group = 'all'

scores = cross_validate(pipe, X_cv, y_cv, cv=rkf, scoring=regression_scorer, n_jobs=-1, 
                        return_train_score=True, return_estimator=True)

coefs = np.array([estimator['plsregression'].coef_ for estimator in scores['estimator']])
coefs = np.squeeze(coefs, axis=2)
avg_coef = np.mean(coefs, axis=0)

print(f'pls_{population}_{selected_target}_{age_group}')
for metric in scoring:
    metric_values = scores[metric]
    print(f'Avg {metric}: {np.mean(metric_values):.2f}')

pls_adhd_WISC_PSI_all
Avg train_score: 0.85
Avg test_score: 0.11
CPU times: user 1.43 s, sys: 229 ms, total: 1.66 s
Wall time: 23.1 s


In [30]:
from os.path import join
from common.paths import PLS_WEIGHTS

filename = f'pls_{population}_{selected_target}_{age_group}.npy'

np.save(join(PLS_WEIGHTS, filename), avg_coef)

#### ICC

In [18]:
%%time
import pingouin as pg

icc_data = pd.DataFrame(coefs).melt(var_name='connection', value_name='weight', ignore_index=False)
icc_data['cv_run_num'] = icc_data.index
icc = pg.intraclass_corr(data=icc_data, targets='connection', raters='cv_run_num', ratings='weight').round(3)
icc.set_index("Type")

KeyboardInterrupt: 

### Run permutation statistic

In [23]:
%%time
# X_cv = bin_1[0]
# y_cv = bin_1[1]
X_cv = X
y_cv = y
age_group = 'all'

score, _, pvalue = permutation_test_score(
    pipe, X_cv, y_cv, cv=rkf, scoring=regression_scorer, n_permutations=n_perm, n_jobs=-1)

# print(f'Model: PLS | Group: {population} | Target: {selected_target} | Age: {age_group}')
print(f'pls_{population}_{selected_target}_{age_group}')
print(f'Score: {score:.2f} | p-value: {pvalue:.4f}')

pls_healthy_WISC_PSI_all
Score: -0.06 | p-value: 0.6934
CPU times: user 2min 9s, sys: 4.18 s, total: 2min 13s
Wall time: 4h 47min 31s


## Run for all targets, all age bins

In [11]:
%%time
from common.binning import bin_by_age
from common.wisc import FSIQ, PRIMARY_INDICES
from common.paths import PLS_RESULTS

results = []
targets = FSIQ + PRIMARY_INDICES

for target in targets:
    y = Y[target]
    bins = bin_by_age(X, y, ages, y)
    bin_1, bin_2, bin_3 = bins[0], bins[1], bins[2]
    X_all = [X, bin_1[0], bin_2[0], bin_3[0]]
    y_all = [y, bin_1[1], bin_2[1], bin_3[1]]
    bin_labels = ["All", "Bin 1", "Bin 2", "Bin 3"]
#     X_all = [X]
#     y_all = [y]
#     bin_labels = ["All"]
    
    for X_cv, y_cv, bin_label in zip(X_all, y_all, bin_labels):
        # Do permutation test
        score, _, pvalue = permutation_test_score(
            pipe, X_cv, y_cv, cv=rkf, scoring=regression_scorer, n_permutations=n_perm, n_jobs=-1)
        results.append({    
            'Model': 'pls',
            'Population': population,
            'Target': target,
            'Bin': bin_label,
            'Score': score,
            'P-value': pvalue,
        })
        print(results[-1])

results_df = pd.DataFrame(results)
display(results_df)
results_df.to_csv(PLS_RESULTS)

{'Model': 'pls', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'Bin': 'All', 'Score': 0.35306387905247083, 'P-value': 0.0003332222592469177}
{'Model': 'pls', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'Bin': 'Bin 1', 'Score': 0.15173929020086982, 'P-value': 0.09530156614461846}
{'Model': 'pls', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'Bin': 'Bin 2', 'Score': 0.3601675378581875, 'P-value': 0.0003332222592469177}
{'Model': 'pls', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'Bin': 'Bin 3', 'Score': 0.10843605610181313, 'P-value': 0.1789403532155948}
{'Model': 'pls', 'Population': 'adhd', 'Target': 'WISC_VSI', 'Bin': 'All', 'Score': 0.3293572693285182, 'P-value': 0.0003332222592469177}
{'Model': 'pls', 'Population': 'adhd', 'Target': 'WISC_VSI', 'Bin': 'Bin 1', 'Score': 0.2052393888219043, 'P-value': 0.03632122625791403}
{'Model': 'pls', 'Population': 'adhd', 'Target': 'WISC_VSI', 'Bin': 'Bin 2', 'Score': 0.19724862318696254, 'P-value': 0.02465844718427191}
{'Model': 'pls', 'Populat

,Model,Population,Target,Bin,Score,P-value
0,pls,adhd,WISC_FSIQ,All,0.353064,0.000333
1,pls,adhd,WISC_FSIQ,Bin 1,0.151739,0.095302
2,pls,adhd,WISC_FSIQ,Bin 2,0.360168,0.000333
3,pls,adhd,WISC_FSIQ,Bin 3,0.108436,0.178940
4,pls,adhd,WISC_VSI,All,0.329357,0.000333
5,pls,adhd,WISC_VSI,Bin 1,0.205239,0.036321
6,pls,adhd,WISC_VSI,Bin 2,0.197249,0.024658
7,pls,adhd,WISC_VSI,Bin 3,0.165631,0.071310
8,pls,adhd,WISC_VCI,All,0.368599,0.000333
9,pls,adhd,WISC_VCI,Bin 1,0.162318,0.084972


FileNotFoundError: [Errno 2] No such file or directory: '/home/bpho/Documents/MSc_Research-Project/scratch_data/model_results/pls.csv'